In [3]:
# !pip install ruwordnet
# !ruwordnet download

downloading a ruwordnet model from https://github.com/avidale/python-ruwordnet/releases/download/0.0.4/ruwordnet-2021.db


In [ ]:
from ruwordnet import RuWordNet
from collections import deque
import numpy as np

In [ ]:
# Initialize RuWordNet
ruwordnet = RuWordNet()

In [ ]:
# Function to get all hypernyms for a given synset
def get_hypernyms(synset):
    return synset.hypernyms

In [ ]:
# Bidirectional BFS to find the shortest path between two synsets
def bidirectional_bfs(synset1, synset2):
    if synset1 == synset2:
        return 0
    
    queue1 = deque([(synset1, 0)])
    queue2 = deque([(synset2, 0)])
    
    visited1 = {synset1: 0}
    visited2 = {synset2: 0}
    
    while queue1 and queue2:
        if queue1:
            current_synset, distance = queue1.popleft()
            for hypernym in get_hypernyms(current_synset):
                if hypernym in visited2:
                    return distance + 1 + visited2[hypernym]
                if hypernym not in visited1:
                    visited1[hypernym] = distance + 1
                    queue1.append((hypernym, distance + 1))
        
        if queue2:
            current_synset, distance = queue2.popleft()
            for hypernym in get_hypernyms(current_synset):
                if hypernym in visited1:
                    return distance + 1 + visited1[hypernym]
                if hypernym not in visited2:
                    visited2[hypernym] = distance + 1
                    queue2.append((hypernym, distance + 1))
    
    return float('inf')

In [ ]:
# Function to calculate the minimal distance between two labels
def calculate_distance(label1, label2):
    synsets_label1 = ruwordnet.get_synsets(label1)
    synsets_label2 = ruwordnet.get_synsets(label2)

    min_distance = float('inf')
    for synset1 in synsets_label1:
        for synset2 in synsets_label2:
            distance = bidirectional_bfs(synset1, synset2)
            if distance < min_distance:
                min_distance = distance
    
    return min_distance if min_distance != float('inf') else None

In [ ]:
# Function to calculate the average distance between labels and ideal_label
def calculate(labels, ideal_label):
    # Calculate distances for correct and wrong labels
    distances = [calculate_distance(ideal_label, label) for label in labels]
    # Filter out None values
    distances = [dist for dist in distances if dist is not None]
    # Compute average distances, treating no distances as infinity
    average_distance = np.mean(distances) if distances else float('inf')
    return average_distance